<a href="https://colab.research.google.com/github/hannari-python/tutorial/blob/master/unreleased_works/Ogawa_Kakei_old.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 下記は小川さんの成果

## データフレームの列の選択

`s` のうち `OL_Expenditure`までの列を選択し、細かい分類の支出情報の列を取り除いてみましょう。
そのためにはデータフレームの `loc` メソッドを使います。

In [ ]:
data = s.loc[:, :'OL_Expenditure']
data

## [記述統計量(英: descriptive statistics value)](https://ja.wikipedia.org/wiki/%E8%A6%81%E7%B4%84%E7%B5%B1%E8%A8%88%E9%87%8F) の確認

記述統計量を確認しデータの範囲や代表的な値がどの程度か確認しましょう。
そのためにはデータフレームの `describe` メソッドを使います。


In [ ]:
data.describe()

## plotly (express) を用いた可視化

ここでは家計調査の中項目を可視化してみましょう。    
まずはバイオリンプロットで全ての項目を可視化します。    
PlotlyExpressとデータフレームを使ってグラフを描画する場合、作成したいグラフの種類の関数に、まずデータフレームを持っている変数名を渡し、そのあと表示したい要素を引数に渡します。    
最初の事例では data の全てのコラムをバイオリンで可視化しました。(かなり表示に時間がかかります)





In [ ]:
import plotly.express as px

In [ ]:
px.violin(data, y=data.columns)

## ヒストグラムの作成
続いて Y_Income (収入)のヒストグラムを作成します。     
引数nbins には、ヒストグラムのビンの数を指定 、引数histnormにはパーセントを指定すると、y軸が%表記となります。

In [ ]:
px.histogram(data, x='Y_Income', histnorm='percent', nbins=100)

## ビンを作る
自動でビンを作るほか、自作することもできます。    
pandasのcut関数を使うと、引数binsに指定した数のビンを作成してくれます。     
しかし、この方法は次に見るように、データのテールが長いと使い勝手の悪いビン分けとなります。

In [ ]:
data_bins = pd.cut(data['Y_Income'].values, bins=10)
data_bins

## 自分で作成したビンを割り当てる
numpyのdigitize関数を使うと、自分の指定するビンを作成することができます    
引数x（第1引数）にはビン分けしたい値を、引数bins（第2引数）にビン分けの定義をわたします      
返り値はどのビンに入っているかを数値となります


In [ ]:
import numpy as np 
bin_image = [0, 2000, 4000, 6000, 8000, 12000, 20000]
bin_array = np.digitize(data['Y_Income'], bin_image)

### ビンの割り当て
全てのデータがどのビンに入っているか、割り当てられる

In [ ]:
len(bin_array)

In [ ]:
pd.Series(bin_array).unique()

In [ ]:
help(np.digitize)

## データの列名と項目の確認
データの詳細が書かれたエクセルファイルを読み込み、確認します

In [ ]:
cont = pd.read_excel("/content/ippan_2009zensho_s/ippan_2009zensho_s.xls")

In [ ]:
cont[:50]

## 各消費分類の消費支出に対する割合を作成する
各消費分類が全体に占める割合をみるため、そのデータを作成します    
上で確認したようにL_Expenditureが消費額となり、それで全項目を割ると欲しい値が算出できる     
具体的にはapplyメソッドとlambda式を用いて行全体をL_Expenditureで割ります

In [ ]:
data_test = data.loc[: ,'L_Expenditure':'OL_Expenditure']

In [ ]:
data_test

In [ ]:
data_test.apply(lambda x: x/ data_test['L_Expenditure'])

In [ ]:
data_expenditure_ratio = data_test.apply(lambda x: x / data_test['L_Expenditure'])


## 作成した割合を可視化
再びバイオリンプロットで可視化します     
かなりテールが長いものとなっていることが分かります



In [ ]:
px.violin(data_expenditure_ratio, x=data_expenditure_ratio.columns[1:])

## 分析に使うデータを作成します
- まず、data の基礎情報の部分と先ほど作成した消費の割合のデータをpandasのconcat関数を用いて連結します
- 次に年収のビン分けデータを先ほどのデータに加えます

In [ ]:
data_front = data.loc[:, :'Y_Income']
data_front = data_front.drop('Weight',axis=1)
data_preped = pd.concat([data_front, data_expenditure_ratio], axis=1)

In [ ]:
data_preped

In [ ]:
bin_image = [0, 2000, 4000, 6000, 8000, 12000, 20000]
bin_array = np.digitize(data['Y_Income'], bin_image)
data_preped['bins'] = bin_array

In [ ]:
data_preped

# ビンごとに可視化
ビンごとにデータを可視化します    
この場合、PlotlyExpressではなく、plotly graph_objectsを使って可視化しています     
少し複雑な可視化はPlotlyExpressでは表現しきれなくなります    
そのような場合、Plotly Graph Objectsを使います


In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()

for num in data_preped.bins.unique():
  data_preped_num = data_preped[data_preped['bins'] == num]
  fig.add_trace(go.Histogram(x=data_preped_num['Food'], name=f'{num}', histnorm='probability', nbinsx=20))

fig.show()

## Dashを用いてインタラクティブな可視化！

- ビンの個数を動かせる
- ビンの数値を変更できる
- それをヒストグラムで描画できる

In [ ]:
data_for_dash = data_preped.drop('bins', axis=1)

In [ ]:
from jupyter_dash import JupyterDash 
import dash_core_components as dcc 
import dash_html_components as html 

from dash.dependencies import Input, Output, State, ALL

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
                       

                       html.Button(id='my_button', children='Add Input'),
                      
                      html.Div([
                       html.Div([
                       html.Div(id='input_zone', children=[]),
                      ], style={'width': '25%', 'display': 'inline-block', 'verticalAlign': 'top'}),
                       


                       html.Div([
                                 dcc.Dropdown(id='my_dropdown',
                                              options=[{'label': col, 'value': col} for col in data_for_dash.columns],
                                              value='Food'
                                              ),
                                 dcc.Graph(id='my_graph'),
                                
                                html.Div([
                                 html.H3('Histogram Bin Num: '),
                                 dcc.Input(id='bin_num', value=10, type='number'),
                                ]),
                       ], style={'width': '70%', 'display': 'inline-block'}),
                      ]),
                       
                       html.Div([
                       dcc.RangeSlider(id='my_range_slider',
                                       min=0,
                                       max=data_for_dash['Y_Income'].max()
                                       
                                       ),
                        html.Button(id='slider_button', children='graph update'),
                       ], style={'width': '90%', 'height': 100, 'margin': 'auto'}),

])

@app.callback(Output('input_zone', 'children'), [Input('my_button', 'n_clicks')], [State('input_zone', 'children')], prevent_initial_call=True)
def update_input_zone(n_clicks, existing_children):
  my_inputs = html.Div([
                        dcc.Input(id={'type': 'my_inputs', 'index': n_clicks}, value=0)
  ])
  existing_children.append(my_inputs)
  return existing_children

@app.callback(Output('my_range_slider', 'value'), [Input({'type': 'my_inputs', 'index': ALL}, 'value')])
def update_slider(enter_values):
  if len(enter_values) > 1:
    enter_values = [int(i) for i in enter_values]
  return enter_values

@app.callback(Output('my_graph', 'figure'), [Input('slider_button', 'n_clicks'), Input('my_dropdown', 'value'), Input('bin_num', 'value')], [State('my_range_slider','value')], prevent_initial_call=True)
def update_graph(n_clicks, slider_values, bin_num, selected_values):

  if len(selected_values) > 1 and sum(selected_values) > 1:
    bin_array = np.digitize(data_for_dash['Y_Income'], selected_values)
    data_for_dash['bins'] = bin_array  
    fig = go.Figure()
    for num in data_for_dash['bins'].unique():
      update_df = data_for_dash[data_for_dash['bins'] == num]
      fig.add_trace(go.Histogram(x=update_df[slider_values], histnorm='probability', nbinsx=bin_num))
  
    return fig
  return dash.no_update


app.run_server(mode='inline')